# Cobre 

In [ ]:
import os
import pandas as pd
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [ ]:
#Setando var bases
caminho = r"C:\Users\A0161597\OneDrive - Telefonica\Documentos\Rentabilidade\Relatorios\2025\04-ABRIL - Copia\REPOS"

caminhopadrao = r"C:\Users\A0161597\OneDrive - Telefonica\Documentos\Rentabilidade\Relatorios\2025\01-JANEIRO\JANEIRO25.xlsx"

meses = {
    1 : 'Jan',
    2 : 'Fev',
    3 : 'Mar',
    4 : 'Abr',
    5 : 'Mai',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Ago',
    9 : 'Set',
    10 : 'Out',
    11 : 'Nov',
    12 : 'Dez',
}

data = date.today()
ano_peq = date.today().strftime("%y")
data = pd.to_datetime("2025-04-01")

ultimos_12_meses = []

for i in range(0,12):
    ref = data - relativedelta(months=i)
    ref2 = f'{meses[int(ref.strftime("%m"))]}_{ref.strftime("%y")}_Val'
    ultimos_12_meses.append(ref2)

premissa_voz = 4.95
premissa_bl = 10.43

premissa_redext_voz = 4.08483115527778
premissa_redext_bl = 7.51640164583333

In [ ]:
#Carregando arquivos

padrao_file = pd.ExcelFile(caminhopadrao)
padrao = padrao_file.sheet_names

energia_file = pd.ExcelFile(os.path.join(caminho,'ENERGIA.xlsx'))
energia = energia_file.sheet_names

aluguel_df = pd.read_csv(os.path.join(caminho,"ALUGUEL.csv"), sep='|')

receita_df = pd.read_csv(os.path.join(caminho,'RECEITA.txt'), sep='\t', decimal=',',encoding='latin-1')

hades_file = pd.ExcelFile(os.path.join(caminho,'HADES.xlsx'))
hades = hades_file.sheet_names

print(f'\nPadrao:\n{padrao}')
print(f'\nEnergia:\n{energia}')
print(f'\nHades:\n{hades}')

padrao_armario = padrao_file.sheet_names[0]
padrao_central = padrao_file.sheet_names[1]
energia = energia_file.sheet_names[0]
hades = hades_file.sheet_names[2]

padrao_armario_df = padrao_file.parse(padrao_armario)
padrao_central_df = padrao_file.parse(padrao_central)
energia_df = energia_file.parse(energia, header=3)
hades_df = hades_file.parse(hades, header=1)


In [ ]:
#Tratamento Planilha Padrao
#parse todas as planilhas

planilhas_padrao = {
    'BUCLE CENTRAL',
    'REDE EXT CENTRAL',
    'REDE EXT E ACESSO ARM',
    'PREMISSA',
    'DE PARA AREA ARM',
    'DE PARA ENERGIA',
    'DE PARA CENTRAL',
    'DE PARA ARMARIO'
}

#ingestao todas as planilhas
for i in planilhas_padrao:
    var_name = f"df_{i.replace(' ', '_')}"
    globals()[var_name] = padrao_file.parse(i)


df_DE_PARA_ENERGIA = df_DE_PARA_ENERGIA[['COD', 'UF/ED']] # type: ignore
df_DE_PARA_ARMARIO = df_DE_PARA_ARMARIO.iloc[:, 2:8] # type: ignore
df_DE_PARA_AREA_ARM = df_DE_PARA_AREA_ARM[['NOME RENT.', 'AREA']] # type: ignore

df_DE_PARA_AREA_ARM.rename(columns={
    'NOME RENT.':'ARMARIO'
}, inplace=True)

df_REDE_EXT_E_ACESSO_ARM.rename(columns={ # type: ignore
    'MANUT TOTAL':'REDE EXT. + MANUT'
}, inplace=True)



In [ ]:
#Trata planilha ENERGIA para join 

colunas = ['UFED']
colunas = colunas + ultimos_12_meses
energia_df = energia_df[colunas]

energia_df = pd.merge(energia_df, df_DE_PARA_ENERGIA, how='left', left_on='UFED', right_on='UF/ED')

energia_df = energia_df.groupby('COD')[ultimos_12_meses].sum().reset_index()

energia_df['ENERGIA'] = energia_df[ultimos_12_meses].sum(axis=1).div(12)

energia_df = energia_df[['COD', 'ENERGIA']]

energia_df.dropna(subset=['COD'], inplace=True)

energia_df.rename(columns={
    'COD':'ARMARIO'
}, inplace=True)

energia_df.head()

In [ ]:
#Trata planilha aluguel para join 

colunas = ['pca_gp_r011_zrea.bnwhr','pca_gp_r011_zrea.recntxt','pca_gp_r011_zrea.dbervon_s']
aluguel_df = aluguel_df[colunas]

aluguel_df['pca_gp_r011_zrea.recntxt'] = aluguel_df['pca_gp_r011_zrea.recntxt'].str.replace(' ', '')

aluguel_df.rename(columns={
    'pca_gp_r011_zrea.bnwhr':'ALUGUEL',
    'pca_gp_r011_zrea.recntxt':'CONTRATO',
    'pca_gp_r011_zrea.dbervon_s':'PERIODO'
}, inplace=True)

aluguel_df['PERIODO'] = pd.to_datetime(aluguel_df['PERIODO'], format='%Y%m')

aluguel_df = aluguel_df[(aluguel_df['PERIODO'] <= data) & (aluguel_df['PERIODO'] >= (data - relativedelta(months=11)))]

aluguel_df = aluguel_df.groupby('CONTRATO')['ALUGUEL'].sum().div(12).reset_index()

aluguel_df = pd.merge(aluguel_df, df_DE_PARA_ARMARIO, how='left', left_on='CONTRATO', right_on='DE_PARA_ALUGUEL')

aluguel_df = aluguel_df[['ARMARIO', 'ALUGUEL']]

aluguel_df.dropna(subset='ARMARIO', inplace=True)
aluguel_df.count()

In [ ]:
#Trata planilha receita para join

receita_df = receita_df[receita_df['TECNOLOGIA_PRINICIPAL'] == '2 - COBRE']

receita_df = receita_df[['ARMARIO_PRINCIPAL','SEGMENTO', 'Sum(VLR_TOTAL)', 'Sum(QTD_VOZ_MULTI)', 'Sum(QTD_BL_MULTI)']]

receita_df.rename(columns={
    'ARMARIO_PRINCIPAL':'ARMARIO',
    'SEGMENTO':'SEG',
    'Sum(VLR_TOTAL)':'REC BRUTA',
    'Sum(QTD_VOZ_MULTI)':'UGR VOZ',
    'Sum(QTD_BL_MULTI)':'UGR BL',
}, inplace=True)

receita_df['REC BRUTA'] = receita_df['REC BRUTA'].str.replace('.','').str.replace(',','.')
receita_df['REC BRUTA'] = pd.to_numeric(receita_df['REC BRUTA'], errors='coerce')

receitab2b_df = receita_df[receita_df['SEG'] == 'B2B']
receitab2c_df = receita_df[receita_df['SEG'] == 'B2C']

receita_df = receita_df.groupby('ARMARIO').sum().reset_index()
receitab2b_df = receitab2b_df.groupby('ARMARIO').sum().reset_index()
receitab2c_df = receitab2c_df.groupby('ARMARIO').sum().reset_index()

receitab2b_df.rename(columns={
    'REC BRUTA':'REC BRUTA B2B',
    'UGR VOZ':'UGR VOZ B2B',
    'UGR BL':'UGR DADOS B2B',
},inplace=True)

receitab2c_df.rename(columns={
    'REC BRUTA':'REC BRUTA B2C',
    'UGR VOZ':'UGR VOZ B2C',
    'UGR BL':'UGR DADOS B2C',
},inplace=True)

receita_df.drop(['SEG'], axis=1, inplace=True)
receitab2c_df.drop(['SEG'], axis=1, inplace=True)
receitab2b_df.drop(['SEG'], axis=1, inplace=True)


In [ ]:
#Trata planilha hades para join

hades_df = hades_df[['Armários', 'Status']]
hades_df.rename(columns={
    'Armários': 'ARMARIO',
    'Status':'HADES'
}, inplace=True)

In [ ]:
armario_df = padrao_armario_df[['TIPO', 'ARMARIO', 'COD_IBGE', 'DDD', 'REGIONAL','UF', 'CIDADE']]

armario_df = pd.merge(armario_df, df_DE_PARA_AREA_ARM, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, hades_df, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, receita_df, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, receitab2b_df, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, receitab2c_df, how='left', on='ARMARIO')

cond_hades = (armario_df['HADES'] != 'Desligado').astype(int)
cond_tipo = (armario_df['TIPO'] != 'SP').astype(int)

armario_df = pd.merge(armario_df, df_REDE_EXT_E_ACESSO_ARM[['AREA', 'REDE EXT. + MANUT']], how='left', on='AREA') # type: ignore

armario_df['REDE EXT. + MANUT'] = cond_hades * cond_tipo * armario_df['REDE EXT. + MANUT'].fillna(0)

armario_df['SERVIR'] = (armario_df['UGR BL'] * premissa_bl) + (armario_df['UGR VOZ'] * premissa_voz)

armario_df = pd.merge(armario_df, aluguel_df, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, energia_df, how='left', on='ARMARIO')

suporte_df = pd.merge(df_DE_PARA_ARMARIO[['COD BU', 'ARMARIO']], df_BUCLE_CENTRAL[['COD', 'Acesso RTB', 'Acesso ADSL']],how='left', left_on='COD BU', right_on='COD'
)

suporte_df = pd.merge(suporte_df, armario_df[['ARMARIO', 'AREA']], how='left' ,on='ARMARIO')

suporte_df = pd.merge(suporte_df, df_REDE_EXT_E_ACESSO_ARM[['AREA','NETLIS BL MET','NETLIS VOZ MET']], how='left', on='AREA') # type: ignore
                      


suporte_df.drop('AREA', axis=1, inplace=True)

armario_df = pd.merge(armario_df, suporte_df, how='left', on='ARMARIO')

armario_df['ACESSO'] = (
    np.where(
        armario_df['UF'] == 'SP',
        np.where(armario_df['Acesso RTB'] != 0, armario_df['Acesso RTB'], premissa_redext_voz),
        np.where(armario_df['NETLIS VOZ MET'] != 0, armario_df['NETLIS VOZ MET'], premissa_redext_voz)
    ) * armario_df['UGR VOZ']
) + (
    np.where(
        armario_df['UF'] == 'SP',
        np.where(armario_df['Acesso ADSL'] != 0, armario_df['Acesso ADSL'], premissa_redext_bl),
        np.where(armario_df['NETLIS BL MET'] != 0, armario_df['NETLIS BL MET'], premissa_redext_bl)
    ) * armario_df['UGR BL']
)

armario_df.drop(['COD BU', 'COD', 'Acesso RTB', 'Acesso ADSL', 'NETLIS BL MET', 'NETLIS VOZ MET'], axis=1, inplace=True)


In [ ]:
central_df = padrao_central_df[['Central','AT','COD','Município','Estação/Armário','Sobreposicao']]

central_df.rename(columns={
    'Central':'CENTRAL'
}, inplace=True)

central_df = pd.merge(central_df, receita_df, how='left', left_on='CENTRAL', right_on='ARMARIO')

central_df = pd.merge(central_df, receitab2b_df, how='left', on='ARMARIO')

central_df = pd.merge(central_df, receitab2c_df, how='left', on='ARMARIO')

central_df = pd.merge(central_df, df_REDE_EXT_CENTRAL[['CENTRAL', 'UNITÁRIO POR AT', 'CONTRATADA']], how='left', on='CENTRAL')

suporte_df = pd.merge(central_df[['CENTRAL', 'CONTRATADA']], df_BUCLE_CENTRAL[['NOME 2', 'Acesso RTB', 'Acesso ADSL']], how='left', left_on='CONTRATADA',right_on='NOME 2')

suporte_df = suporte_df[['CENTRAL', 'Acesso RTB', 'Acesso ADSL']]

central_df = pd.merge(central_df, suporte_df, how='left', on='CENTRAL')

central_df['ACESSO'] = (central_df['Acesso ADSL'] * central_df['UGR BL']) + (central_df['Acesso RTB'] * central_df['UGR VOZ'])

central_df = pd.merge(central_df, energia_df, how='left', on='ARMARIO')

central_df['SERVIR'] = (central_df['UGR VOZ'] * premissa_voz) + (central_df['UGR BL'] * premissa_bl)

In [ ]:
df_BUCLE_CENTRAL['NOME 2'].head()


In [ ]:
energia_df.head()

In [ ]:
central_df.head()

In [ ]:
pd.set_option('display.max_columns', None)
armario_df.head()

In [ ]:
central_df.head()

pd.set_option('display.max_columns', None)

subtotal = central_df.sum(numeric_only=True)

formatted_subtotal = subtotal.apply(lambda x: f"{x:,.2f}")

# Exibir
print(formatted_subtotal)
